<a href="https://colab.research.google.com/github/alihammadbaig/pdf_to_excel/blob/master/pdf_to_excel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
pip install tika

  Stored in directory: /root/.cache/pip/wheels/b4/db/8a/3a3f0c0725448eaa92703e3dda71e29dc13a119ff6c1036848
Successfully built tika


In [0]:
# from shutil import copyfile

# copyfile("/tika-server-1.18.jar", "/tmp//tika-server.jar")

In [0]:
# os.environ["TIKA_PATH"] = "/tmp/tika-server-1.18.jar"
# os.environ["TIKA_SERVER_JAR"] = "/tmp/tika-server-1.18.jar"
# print(os.environ["TIKA_SERVER_JAR"])


In [0]:
import os
os.environ['TIKA_SERVER_JAR'] = 'https://repo1.maven.org/maven2/org/apache/tika/tika-server/1.19/tika-server-1.19.jar'
import tika
from tika import parser

In [0]:
# Text parsing library
# import tika
# from tika import parser

# Deal with regular expression(text processing)
import re

# Handle the excel spreadsheet
import openpyxl

import os

tika.initVM()

# Read the ;PDF file
# parsed = parser.from_file('/content/gdrive/My Drive/Colab Notebooks/data/sample_pdf_for_excel.pdf')
# parsed = parser.from_file('/content/gdrive/My Drive/Colab Notebooks/data/20files/L290-AQ-MDR-13850_0_SECTN_100OF107_IPX Reviewed_AC-OCR.pdf')
# parsed = parser.from_file('/content/gdrive/My Drive/Colab Notebooks/data/20files/L290-AQ-MDR-13850_0_SECTN_84OF107_IPX Reviewed_AC-OCR.pdf')
# parsed = parser.from_file('/content/gdrive/My Drive/Colab Notebooks/data/20files/L290-AQ-MDR-13850_0_SECTN_100OF107_IPX Reviewed_AC-OCR.pdf')
# parsed = parser.from_file('/content/gdrive/My Drive/Colab Notebooks/data/20files/L290-AQ-MDR-13850_0_SECTN_91OF107_IPX Reviewed_AC-OCR.pdf')
# parsed = parser.from_file('/content/gdrive/My Drive/Colab Notebooks/data/20files/L290-AQ-MDR-13850_0_SECTN_92OF107_IPX Reviewed_AC-OCR.pdf')
# parsed = parser.from_file('/content/gdrive/My Drive/Colab Notebooks/data/20files/L290-AQ-MDR-13850_0_SECTN_101OF107_IPX Reviewed_AC-OCR.pdf')
# parsed = parser.from_file('/content/gdrive/My Drive/Colab Notebooks/data/20files/L290-AQ-MDR-13850_0_SECTN_102OF107_IPX Reviewed_AC-OCR.pdf')
# parsed = parser.from_file('/content/gdrive/My Drive/Colab Notebooks/data/20files/L290-AQ-MDR-13850_0_SECTN_93OF107_IPX Reviewed_AC-OCR.pdf')
# parsed = parser.from_file('/content/gdrive/My Drive/Colab Notebooks/data/20files/L290-AQ-MDR-13850_0_SECTN_89OF107_IPX Reviewed_AC-OCR.pdf')
# parsed = parser.from_file('/content/gdrive/My Drive/Colab Notebooks/data/20files/L290-AQ-MDR-13850_0_SECTN_94OF107_IPX Reviewed_AC-OCR.pdf')
# parsed = parser.from_file('/content/gdrive/My Drive/Colab Notebooks/data/20files/L290-AQ-MDR-13850_0_SECTN_96OF107_IPX Reviewed_AC-OCR.pdf')
# parsed = parser.from_file('/content/gdrive/My Drive/Colab Notebooks/data/20files/L290-AQ-MDR-13850_0_SECTN_90OF107_IPX Reviewed_AC-OCR.pdf')
# parsed = parser.from_file('/content/gdrive/My Drive/Colab Notebooks/data/20files/L290-AQ-MDR-13850_0_SECTN_85OF107_IPX Reviewed_AC-OCR.pdf')
# parsed = parser.from_file('/content/gdrive/My Drive/Colab Notebooks/data/20files/L290-AQ-MDR-13850_0_SECTN_103OF107_IPX Reviewed_AC-OCR.pdf')
# parsed = parser.from_file('/content/gdrive/My Drive/Colab Notebooks/data/20files/L290-AQ-MDR-13850_0_SECTN_104OF107_IPX Reviewed_AC-OCR.pdf')
# parsed = parser.from_file('/content/gdrive/My Drive/Colab Notebooks/data/20files/L290-AQ-MDR-13850_0_SECTN_105OF107_IPX Reviewed_AC-OCR.pdf')
parsed = parser.from_file('/content/gdrive/My Drive/Colab Notebooks/data/20files/L290-AQ-MDR-13850_0_SECTN_106OF107_IPX Reviewed_AC-OCR.pdf')


# parsed = parser.from_file('/content/gdrive/My Drive/Inpex files /L290-AQ-MDR-13850_0_SECTN_89OF107_IPX Reviewed_AC-OCR.pdf')
content = parsed["content"].strip()

# Read the spreadsheet for later use
wb = openpyxl.load_workbook("/content/gdrive/My Drive/Colab Notebooks/data/20files/L290-AQ-MDR-13850_0_SECTN_106OF107_IPX Reviewed_AC-OCR.xlsx")
sheet = wb['Template']

# Starting writing the sheet beginning line#2
sheet_row_num = 2

# PDF columns
col_names = ["top_coat_date", "blast_primer_date"
             ,"dft1_ave", "dft1_5", "dft1_4", "dft1_3", "dft1_2", "dft1_1"
             ,"dft2_ave", "dft2_5", "dft2_4", "dft2_3", "dft2_2", "dft2_1"
             , "sys_tem"]

# Spreadsheet cells
cells = ["AL", "S", "AC", "AB", "AA", "Z", "Y", "X", "K", "J", "I", "H", "G", "F", "D"]

cols = {}

In [20]:
page_num_printed = 13322
page_num = 1

total_pages = int(parsed["metadata"]["xmpTPg:NPages"].strip())
# total_pages = 5
# print(total_pages)

def get_col():
  global r
  r = r.strip()
  val = r[r.rfind(" ") : len(r)]
  r = r[0 : r.rfind(" ")]
  return val

def hasNumbers(inputString):
  return bool(re.search(r'\d', inputString))

import traceback

'''
## Pages ##
'''
# print(page_num)
# print(total_pages)

while True:
  skip = 11
# for page_num in range(1, total_pages):
  print("Page#%i"%page_num)
  print("Page num printed#%i"%page_num_printed)
  
  start_of_page = "Page " + str(page_num_printed) +" of 13699"
  end_of_page = "Page " + str(page_num_printed + 1) +" of 13699"
  page_content = content[content.find(start_of_page): content.find(end_of_page)]
  
#   print("----- Start: Page Content of page:{} -----".format(page_num_printed))
#   print(page_content)
#   print("----- End: Page Content of page:{} -----".format(page_num_printed))
  
  ## Extract the table
  try:
    p = re.compile("(A*VE\\s+DATE)", re.IGNORECASE)
    for m in p.finditer(page_content):
      start_of_table = m.start()+8

    end_of_table = re.search("(FINAL\\s+)*INSPECTION", page_content, flags = re.I)
#     print("1-End of table:{}".format(end_of_table))
    if end_of_table is None:
      end_of_table = re.search("(VISUAL\\s+)*INSPECTION", page_content, flags = re.I)
      skip = 29
#       print("2-End of table:{}".format(end_of_table))
    
    if end_of_table is None: 
      end_of_table = re.search("\\(*REMARK", page_content, flags = re.I)
      skip = 50
#       print("3-End of table: {}".format(end_of_table))
    
    end_of_table = end_of_table.start() - skip
#     print("End of table:{0} and skip:{1}".format(end_of_table,skip))
    
    table = page_content[start_of_table:end_of_table]
#     print(table)
    
    # Data cleansing
    table = table.replace("·","-")\
      .replace("•","")\
      .replace(" ' "," ")\
      .replace("l4", "14")\
      .replace(",", "")\
      .replace(" . ", " ")\
      .replace(" - ", " ")
    
    table = re.sub("OS-([A-Z]{3})", r"05-\1", table,  flags=re.I)
    table = re.sub("(\d{1,2})\.*-\.*([A-Z]{3})\.*-\.*(\d{2,4})", r"\1-\2-\3", table,  flags = re.I)
    table = re.sub("\s+(\d{1,2})\s*-\s*([A-Z]{3})\s*-\s*(\d{2,4})", r" \1-\2-\3 ", table, flags = re.I)
    table = re.sub("\s+([A-Z0-9]{1,2})\s*-\s*([A-Z]{3})\s*-\s*(\d{2,4})", r" \1-\2-\3 ", table, flags = re.I)
    table = re.sub("\s+([A-Z0-9]{1,2})\.*-\.*([A-Z]{3})\.*-\.(\d{2,4})", r" \1-\2-\3 ", table, flags = re.I)
    table = re.sub("\s+OS\s*-\s*([A-Z]{3})\s*-\s*(\d{2,4})", r" 05-\1-\2 ", table, flags = re.I)
    table = re.sub("\s+Ol\s*-\s*([A-Z]{3})\s*-\s*(\d{2,4})", r" 01-\1-\2 ", table, flags = re.I)
    table = re.sub("\-No\>J\-", r"-Nov-", table, flags = re.I)
    table = re.sub("\.+\s*Qct", r"-Oct", table, flags = re.I)
    table = re.sub("12S|l2S|l25", r"125", table)
    table = re.sub("lOS|l0S|l05|1OS|10S", r"105", table)
    table = re.sub(" 19-Nov ", "19-Nov-14", table)
    table = re.sub("(\d\d)\-(.*)?\-(\d\d)", r"\1-\2-\3", table)
    table = re.sub("(0l!!c)|(0i!!c)", "Dec", table)
    table = re.sub("\-0ec.*?\-", "-Dec-", table)
    table = re.sub("\-Oec\-", "-Dec-", table) 
    table = re.sub("(\d\d)\-([A-Z]{3})\s+", r"\1-\2-9999", table, flags = re.I)
    table = re.sub("(\d\d)\-\s*([A-Z]{3})\s*\-\s*(\d\d)", r"\1-\2-\3", table, flags = re.I)
    table = re.sub("\s*l\s*S", "15", table)
    table = re.sub("\s*l\s*O\s*", "10", table)
    table = re.sub("OB", r"08", table, flags = re.I)
    table = re.sub("f|j\s*a\s*n", "Jan", table, flags = re.I)
    table = re.sub("O!Han", "01-Jan", table, flags = re.I)
    table = re.sub("0i!!!c", "Jan", table, flags = re.I)
    table = re.sub("\d\-\d\s*\-[a-z]{3}\-14", "27-Dec-14", table, flags = re.I)
    table = re.sub("\-vi", "-14", table, flags = re.I)
    table = re.sub("(\d\d)(\d\d)+\-(\d\d)\-(\d\d*)", r"\1-Dec-\3", table, flags = re.I)
    table = re.sub("(\d\d)(\d\d)+\-([a-z]){2,}\-(\d\d)", r"\1-\2-\3", table, flags = re.I)
    table = re.sub("(\d\d)(\d\d)+\-(\d\d)(\.|\:|,|;)*\-(\d\d*)", r"\1-Dec-\3", table, flags = re.I)
    table = re.sub("(\d\d)\-([a-z]{2,})(\-\d{2,})+", r"\1-Dec-14", table, flags = re.I)
    table = re.sub("\-\s*0\s*e\s*c\s*\-", "-Dec-", table, flags = re.I)
    table = re.sub("Dcc", "Dec", table, flags = re.I)
    table = re.sub("([a-z]{3})\-1", r"\1-14", table, flags = re.I)
    table = re.sub("([a-z]{3})\-144", r"\1-14", table, flags = re.I)
    table = re.sub("De", "Dec", table, flags = re.I)
    table = re.sub("Decc", "Dec", table, flags = re.I)
    table = re.sub("(\d+)\-Dec$", r"\1-Dec-14", table, flags = re.I)
    table = re.sub("\:\-", "-", table, flags = re.I)
    table = re.sub("Janan", "Jan", table, flags = re.I)
    table = re.sub("1\s*\.*S|l\.*\s*S|\]\s*\.*S", "15", table, flags = re.I)
    table = re.sub("2S", "25", table, flags = re.I)    
    table = re.sub("os\.|oa\.", "05-", table, flags = re.I)
    table = re.sub("J\.i\:n|J\s*\.*;\s*\.*m|J\s*\.*l\s*\.*n|j\s*\.*t\s*\.*n|j\s*\.*t\s*\.*m|j;*:*m|j:*;*m|jo1n|j01n|join|joln|j0ln", "Jan", table, flags = re.I)
    table = re.sub("l\s*\.*l", "11", table, flags = re.I)
    table = re.sub("l(\d)", r"1\1", table, flags = re.I)
    table = re.sub("(\d)l", r"\1(1)", table, flags = re.I)
    table = re.sub(">|<|\s*..._\s*|\-n", "", table, flags = re.I)
    table = re.sub("([a-z]{3})(\d\d)", r"\1-\2", table, flags = re.I)
    table = re.sub("\-(\d{3,})", "14", table)
    ## more rules
    table = re.sub("\-([a-z]{3})(\d{2})", r" \1-\2", table, flags = re.I)
    table = re.sub("\s(\d+)\-Au", r" \1-Aug-14", table, flags = re.I)
    table = re.sub("\s(\d+)\-Ju", r" \1-Jun-14", table, flags = re.I)
#     table = re.sub("\s(\d+)\-Ap", r" \1-Apr-14", table, flags = re.I)
    table = re.sub("\s(\d+)\s+([A-Z]{3})\-(\d\d)", r" \1-\2-\3", table, flags = re.I)
    table = re.sub("\sB\s*\-*([A-Z]{3})\-(\d\d)", r" 08-\1-\2", table, flags = re.I)
    table = re.sub("\sg\s*\-*\d\d", "", table, flags = re.I)
    table = re.sub("\dg\s*\-*\s*(\d\d)", r"01-Aug-\1", table, flags = re.I)
    table = re.sub("\sS\s*\-*([A-Z]{3})\-(\d\d)", r" 05-\1-\2", table, flags = re.I)
    table = re.sub("\s(\d)\s*\-*\s*([A-Z]\s*[A-Z]\s*[A-Z])\s*\-*\s*(\d\d)", r" \1-\2-\3", table, flags = re.I)
    table = re.sub("\-*\s*([A-Z]\s*[A-Z]\s*[A-Z])\s*\-*\s*", r"-\1-", table, flags = re.I)
    table = re.sub("(\d)\s*\-\s*A ug\-(\d\d)", r"\1-Aug-\2", table, flags = re.I)
    table = re.sub("(\d)\s*\-\s*([A-Z]{3})\-(\d\d)\\\-\d\d", r"\1-\2-\3", table, flags = re.I)
    table = re.sub("\-[a-z]+(\d\d)", r"-\1", table, flags = re.I)
    table = re.sub("\-\s*J\s*U\s*L\s*\-|\-\s*J\s*\-|\-\s*U\s*L\s*\-", "-Jul-", table, flags = re.I)
#     table = re.sub("\-\s*M\s*A\s*Y\s*\-|\-\s*M\s*A\s*\-|\-\s*A\s*Y\*\-|\-\s*M\s*\-", "-May-", table, flags = re.I)
    table = re.sub("\s+a\s*y\s*-\s*(\d+)\s*", r"01-May-\1", table, flags = re.I)
    table = re.sub("\s+l\s*-\s*(\d+)\s*", r"01-Jul-\1", table, flags = re.I)
    table = re.sub("(\d)r\-\d+", r"\1", table)
    
    
    '''
    ## Rows ##
    '''
    for row in [x for x in table[2:].splitlines() if x]:
      row = row.strip()
      row = re.sub("-\\s*$", "", row)
      row = re.sub("^.*?\\s+", "", row)
      row = re.sub("^\\s*\\w+\\s+", "", row)
      row = re.sub("\s+[0-9-_]+$", "", row)
      row = re.sub("\s+", " ", row)
      row = re.sub("\-+\s*\~+\s*\~|\~+|\~+\s*\-+|\-\s*$|\~\s*$|\s+[a-z]+$|\s+[a-z]+\_+\s*$|\d+\_+$|\d+\-+$", " ", row)
      row = re.sub("\"|\'", "", row, flags = re.I)
      row = re.sub("(\d+)\-([a-z]+)$\-*", r"\1-\2-14", row, flags = re.I)
      row = re.sub("\-J\-", "Jun", row, flags = re.I)
      row = re.sub("\-\s*Ma\s*\-|\-\s*M\s*\-|BMay", "May", row, flags = re.I)
      row = re.sub("(\d+)([A-Z]{3})(\d{2,})", r"\1-\2-\3", row, flags = re.I)
      row = re.sub("M01-May-1", r"May-14", row, flags = re.I)
      row = re.sub("S", "5", row, flags = re.I)
      row = re.sub("OS", "05", row, flags = re.I)
      row = re.sub("SO", "50", row, flags = re.I)
      row = re.sub("(\d+)D", r"\g<1>0", row, flags = re.I)
      row = re.sub("\s+\-\s*([A-Z]{3})\s*\-\s*(\d{2})", r"01-\1-\2", row, flags = re.I)
      row = re.sub("\d\(\d\)", "01", row, flags = re.I)
      row = re.sub("\-([A-Z]{3})\s*\-\s*[A-Z]+", r"\1-14", row, flags = re.I)
      row = re.sub("\-\s*(\d+)l\s*\-\s*\d+", r"-\1", row, flags = re.I)
      row = re.sub("(\d+)o", r"\1", row, flags = re.I)
      row = re.sub("!0", "10", row)
      row = re.sub("(\d)ov", r"\1-Nov-", row, flags = re.I)
      row = re.sub("(\d+)S", r"\1", row, flags = re.I)
      row = re.sub("l", "1", row, flags = re.I)
      row = re.sub("\.+|\!|\/|\:|i|r", "", row, flags = re.I)
      row = re.sub("J4|l4", "14", row, flags = re.I)
      row = re.sub("4l", "41", row, flags = re.I)
      row = re.sub("D(\d)", r"0\1", row, flags = re.I)
      row = re.sub("2e1", "21", row, flags = re.I)
      row = re.sub("2e1", "21", row, flags = re.I)
     
      
      row = re.sub("\s+\)|\s+\d+\s+\\|\s*v\s*|bl5|\-*eb|\s+1\-15", "", row, flags = re.I)
      row = re.sub("bl5|b14|bl4", "", row, flags = re.I)
                 
            
      # Very specific rules for one file
      row = re.sub("11-1ul-16-Dec-14", "11-Jul-14 16-Dec-14", row, flags = re.I)
      row = re.sub("15-117-Jun-14", "15-Jul-14 17-Jun-14", row, flags = re.I)
      row = re.sub("227-Sov-14v14", "22-Nov-14 01-Nov-14", row, flags = re.I)
      row = re.sub("(\d+)\s*v\s*\-*\s*(\d+)", r"\1-Nov-\2", row, flags = re.I)
      row = re.sub("l\-Dec\-14\-Dec\-15", "01-Dec-14 01-Dec-15", row, flags = re.I)
      row = re.sub("11\-1ul\-16\-Dec\-14", "11-Jul-16 01-Dec-14", row, flags = re.I)
      row = re.sub("(\d+)V\-15\-Dec\-14", r"\1-Nov-15 01-Dec-14", row, flags = re.I)      
#       row = re.sub("(\d+)\-([A-Z]+)\-(\d+)\s+\d+", r"\1-\2-\3", row, flags = re.I)
      row = re.sub("\\\-16", "", row, flags = re.I)
#       row = re.sub("([A-Z]+)\s*\-\s*(\d+)\s+\d+", r"\1-\2", row, flags = re.I)
      row = re.sub("(\d)(\d{2})\-Sov\-14\-Nov", r"\1-\2-Nov-14 01-Nov", row, flags = re.I)
      row = re.sub("12\-Jun\-14I\-16 l 4l", "12-Jun-14 01-Jul-16", row, flags = re.I)
      row = re.sub("l-Dec-15-Dec-15", "01-Dec-15 01-Dec-15", row, flags = re.I)
      row = re.sub("237\-lDec\-14\-Dec\-15", "237 01-Dec-14 01-Dec-15", row, flags = re.I) 
      row = re.sub("\-Jan\-NA\-14c\s*O", "01-Jan-14", row, flags = re.I) 
      row = re.sub("1G|1A", "10", row, flags = re.I)
      row = re.sub("Zl|2l|2i|Zi|Z1", "21", row, flags = re.I)
      
      # Fix space in month names
      row = re.sub("\s*J\s*\-*\s*A\s*\-*\s*N", "Jan", row, flags = re.I)
      row = re.sub("\s*F\s*\-*\s*E\s*\-*\s*B", "Feb", row, flags = re.I)
      row = re.sub("\s*M\s*\-*\s*a\s*\-*\s*r", "Mar", row, flags = re.I)
      row = re.sub("\s*A\s*\-*\s*P\s*\-*\s*R", "Apr", row, flags = re.I)
      row = re.sub("\s*M\s*\-*\s*A\s*\-*\s*Y", "May", row, flags = re.I)
      row = re.sub("\s*J\s*\-*\s*U\s*\-*\s*N", "Jun", row, flags = re.I)
      row = re.sub("\s*J\s*\-*\s*U\s*\-*\s*L", "Jul", row, flags = re.I)
      row = re.sub("\s*A\s*\-*\s*U\s*\-*\s*G", "Aug", row, flags = re.I)
      row = re.sub("\s*S\s*\-*\s*E\s*\-*\s*P", "Sep", row, flags = re.I)
      row = re.sub("\s*O\s*\-*\s*C\s*\-*\s*T", "Oct", row, flags = re.I)
      row = re.sub("\s*N\s*\-*\s*O\s*\-*\s*V", "Nov", row, flags = re.I)
      row = re.sub("\s*D\s*\-*\s*E\s*\-*\s*C", "Dec", row, flags = re.I)
      
      row = re.sub("\s*M\s*\-*\s*A\s*|\s*M\s*\-*\s*Y\s*|\s*A\s*\-*\s*Y\s*|\s*M\s*\-*\s*O\s-*\s*Y\s*|\s*M\s*\-*\s*\~\s-*\s*Y\s*|\s*M\s*\-*\s*V", "May", row, flags = re.I)
      row = re.sub("Mayy|Mayv", "May", row, flags = re.I)
      row = re.sub("\-\s*AP\s*\-", "Apr", row, flags = re.I)
      row = re.sub("\s*M\s*N\*|\sM\s*i\s*|\s*M\s*N\s*|\s*M\s*U\s*|M\s*\d+\s*Y", "May", row, flags = re.I)
      row = re.sub("\s*M\s*N\*|\sM\s*i\s*|BMay", "May", row, flags = re.I)
      row = re.sub("(\d){2}\-([a-z]{3})\-*\s+", r"\1-\2-14", row, flags = re.I)
      row = re.sub("AU&", "Aug", row, flags = re.I)
      row = re.sub("([A-Z]+)1", r"\1l", row, flags = re.I)
      row = re.sub("0CT|0C|OC", "Oct", row, flags = re.I)
      row = re.sub("\s+0\-([A-Z]{3})", r" 01-\1", row, flags = re.I)
      row = re.sub("\s(\d{2})\-([A-Z]{3})\s*$", r" \1-\2-14", row, flags = re.I)
      row = re.sub("\s(\d)\-([A-Z]{3})\-(\d)$", r" 0\1-\2-14", row, flags = re.I)
      row = re.sub("\-(\d)\-(\d)", r"\1\2", row, flags = re.I)
      row = re.sub("M\-JAN|B\s*JAN|J\s*A\s*N", "JAN", row, flags = re.I)
      row = re.sub("\d{3,}\-([A-Z]{3})", r"1-\1", row, flags = re.I)
      row = re.sub("1ul|lu1|1u1|lui|iul|1ui", "Jul", row, flags = re.I)
      row = re.sub("\-J\s*U\-", "-Jul-", row, flags = re.I)
      row = re.sub("JuJ", "Jul", row, flags = re.I)
      row = re.sub("(\d{2})JU", r"\1-Jul", row, flags = re.I)
      row = re.sub("(\d{2})\-([a-z]{3})\-\s", r"\1-\2-14", row, flags = re.I)
      row = re.sub("\s+(\d+)([A-Z]{3})", r"\1-\2", row, flags = re.I)
      row = re.sub("Octt", "Oct", row, flags = re.I)
      row = re.sub("\-\s*D\s*\-", "-Dec-", row, flags = re.I)      
      row = re.sub("\s(\d+)(\d+)\-([a-z]{3})(\d{2})(\d)", r" \1 \2-\3-\4 \5", row, flags = re.I)
      row = re.sub("\s(\d\d)(\d+)\-([a-z]{3})", r" \1 \2-\3", row, flags = re.I)
      row = re.sub("-Heb-14", "01-Feb-14", row, flags = re.I)
      row = re.sub("(\d+)b", r"\1", row, flags = re.I)
      row = re.sub("(\d{1,2})\s*\-\s*([A-Z]{3})\s*\-\s*(\d{2})", r"\1-\2-\3", row, flags = re.I)
      row = re.sub("Jan\-\s*14", r"Jan-14", row, flags = re.I)
      
      row = re.sub("J4|l4", "14", row, flags = re.I)
      row = re.sub("Ap$", "Apr-14", row, flags = re.I)
      row = re.sub("\-\s+(\d{2})", r"-\1", row, flags = re.I)
      
      
      row = re.sub("(\d+)(d{2})\-([a-z]{3})\-(\d{2})", r"\2-\3-\4", row, flags = re.I)
      row = re.sub("(\d{2})\-([a-z]{3})\-(\d{2})(\d{2})\-([a-z]{3})\-(\d{2})(\d{2})\-([a-z]{3})\-(\d{2})", r"\1-\2-\3 \4-\5-\6", row, flags = re.I) # three date together
      
      row = re.sub("\s\-Ju", "", row, flags = re.I)
      row = re.sub("\s+r", "", row, flags = re.I)
      row = re.sub("\s[a-z]+\-", "1-", row, flags = re.I)
      row = re.sub("(\d)([a-z]{3})l\d", r"\1-\2-14", row, flags = re.I)
      row = re.sub("([A-Z]{3})(\d{2})", r"\1-\2", row, flags = re.I)
      
      row = re.sub("(\d+)\-([A-Z]{3})\-(\d+)\-([A-Z]{3})\-(\d+)(\-[A-Z]{3}\-\d+)*", r"\1-\2-14 \3-\4-16", row, flags = re.I)
      row = re.sub("(\-Jan\-+)*NA\-.*", "", row, flags = re.I) 
      row = re.sub("\s+\d\s*\d+$", "", row)
      row = re.sub("(\d\d)(\d)\-", r" \1 \2-", row, flags = re.I)
      
      row = re.sub(";+", "", row, flags = re.I)
      row = re.sub("\-H\-\d+", "", row, flags = re.I)
      row = re.sub("\s1-15|\s1-14", "", row, flags = re.I)
      row = re.sub("V", "", row, flags = re.I)
      
      row = re.sub("\s\d+$", "", row)
      row = re.sub("\s*\--+", "", row)
      row = re.sub("\s\-d{3}$", "", row)
      row = re.sub("\s+(\d)\-([A-Z]{3})\-$", r"\1-\2-14", row, flags = re.I)
      row = re.sub("t14h\-b", "14", row)
      row = re.sub("\-@[a-z]+$", "", row, flags = re.I)
      row = re.sub("\@[a-z]*", "", row, flags = re.I)
      row = re.sub("([a-z]{3})\-\d+\-\d+", r"\1-15", row, flags = re.I)
      row = re.sub("b11", "15 11", row)
      row = re.sub("(\d{2})\-\d{2}", r"\1", row)
      row = re.sub("\s+1$", "", row)
      row = re.sub("\-+", "-", row)
      row = re.sub("(\d\d\d)(\d\d)\-([A-Z]{3})", r"\1 \2-\3", row, flags = re.I)
      row = re.sub("(\d\d)(\d)\-([a-z]+)", r"\1 \2-\3", row)
      row = re.sub("([a-z])\-(\d\d)\d", r"\1-\2", row, flags = re.I)
      row = re.sub("(\d)([a-z])", r"\1-\2", row, flags = re.I)
      row = re.sub("t 1Hc\-14 1\}|2\-A.*", "", row)
      row = re.sub("(\d\d\d)(\d\d)", r"\1 \2", row)
      row = re.sub("(\-ma\-)|(MJ)", "May", row, flags = re.I)
      
      row = re.sub("\-ja\-", "Jan", row, flags = re.I)
      row = re.sub("(5ep)|(5\s*e\s*p)|(5\-*e\-*p)", "Sep", row, flags = re.I)
      row = re.sub("\-e\-", "Sep", row, flags = re.I)
      row = re.sub("\-No\-|\-ov\-", "Nov", row, flags = re.I)
      row = re.sub("bdec", "Dec", row, flags = re.I)
      row = re.sub("([a-z]{3})\-\s+(\d{2})", r"\1-\2", row, flags = re.I)
      row = re.sub("\s(\d+)\-(\d+)\-([a-z]{3})", r"\1\2-\3", row, flags = re.I)
      row = re.sub("  \d+", "", row)
      row = re.sub("(\d+)([a-z]{3})(\d+)", r"\1-\2-\3", row, flags = re.I)
      row = re.sub("(\d)\-M\s*$", r"\1-May-14", row, flags = re.I)
      row = re.sub("c8", "08", row, flags = re.I)
      row = re.sub("(\d)\-M\s*$", r"\1-May-14", row, flags = re.I)
      row = re.sub("(\d)\-([a-z]{3})$", r"\1-\2-14", row, flags = re.I)
      row = re.sub("\-([1-9])\s*$", r"-\g<1>14", row, flags = re.I)
      row = re.sub("\-0\s*$", r"-14", row, flags = re.I)
      row = re.sub("Aprl", "Apr-1", row, flags = re.I)
      row = re.sub("\s+\-\d+$", "", row)
      row = re.sub("\s*O$", "", row, flags = re.I)
      
      row = re.sub("OH.*", "", row, flags = re.I)
      row = re.sub("(\-a\-)|(\-JANEL\-)|(\-Ha\-)|(\-tl0\-)|(\-tJan\-)|(\-o\-)", "-Jan-", row, flags = re.I)
      row = re.sub("\s(\d+)\-$", "", row, flags = re.I)
      row = re.sub("\-\)", "1", row)
      row = re.sub("(\d)\s*\-\s*(\d+)\s*\-\s*(\d)", r"\1-Jan-\3", row, flags = re.I)
      row = re.sub("\s0\-", "01", row)
      row = re.sub("(\d)([a-z]{3})(\d)", r"\1-\2-\3", row)
      row = re.sub("\-a$", "-Jan-16", row, flags = re.I)
      row = re.sub("(\d)(\d{2})([a-z]{3})", r"\1 \2-\3", row, flags = re.I)
      row = re.sub("\s+(\d+)\-(\d)", r"\1 \2", row, flags = re.I)
      row = re.sub("([a-z]{3})(\d)", r"\1-\2", row, flags = re.I)
      row = re.sub("t\-([A-Z])", r"\1", row, flags = re.I)
      row = re.sub("(\d)\-([a-z]{3})\-(\d{2})\s*(\d|\-|\s)+$", r"\1-\2-\3", row, flags = re.I)
      row = re.sub("(\d\d)(\d\d)\-([a-z]{3})", r"\1 \2-\3", row, flags = re.I)
#       row = re.sub("(\-t0714)|(_2-E_1 55)|(\d\-a)|(\-\s*t\s*\-)|([a-z]\-\s\-.*)|(zal.*)|(-\d+\s*OH.*)\
#         |(Jl)|(\d-\s+\-)|(JAN\-\s+\-.*)|(\s+\d+\+\d+)|(\s\d+\-\s*t.*)|(\s+\d+\-$)|(1\s+\-\s+G.*)|(\d\-H.*)\
#         |((\d)(\d\d)\-.*)|(\d{4,})|(\+\-*)|(50 7\-May1\-1)|(__  \-1 \-)|(1\-D\-JAN)|(1\-Hn)\
#         |(w5.*)|(J1E.*)|(\s*\(\s*)|(\s*\)\s*)|(\s1\s*\d+\s*pr.*)|(\s\d+pr.*)|(\spr.*)|(\-(\d+)\s\d\s.*)|(&)|(\d*pr.*)\
#         |(0t)|(2\-QZ\-.*)|(18\-Dec\-\s+0\-ec)|(\d+\-\d+$)|(\s+0\-K)|(\-144)|(0\-ec)|(\-5\-Jan\-15)\
#         |(0144\s*43\s*A2.*)|(954\s*C4\-\s*014 1\-CE3)|(p.*)|(\-Y.*)|(074\s*A.*)|(13\s*1.*)|(\[)|(\\\)|(b\-)", "", row, flags = re.I)
      row = re.sub("l(\d)", r"1\1", row)
      row = re.sub("l", "1", row, flags = re.I)
            
      row = re.sub("(\-\s*Mu)|(\-\s*Mn)", "-May", row, flags = re.I)
      row = re.sub("(\-\s*M\s*\-)", "-May-", row, flags = re.I)
      row = re.sub("(May\s*\-*\s*$)|(\-Me*$)", "May-16", row, flags = re.I)
      row = re.sub("M\s*\-*\s*(\d+)", r"May-\1", row, flags = re.I)
      row = re.sub("O", '0', row, flags = re.I)
      row = re.sub("(\d)\s*([a-z]{3})", r"\1-\2", row, flags = re.I)
      row = re.sub("\-M0\-", "-May-", row, flags = re.I)
      row = re.sub("(\d)([a-z]{3})(\d)", r"\1-\2-\3", row, flags = re.I)
      row = re.sub("\-1\-15", "-15", row)
      row = re.sub("D1", "-Dec-", row)
      
      row = re.sub("\-+", '-', row)
      row = re.sub("(Jn)|(J n)|(b1)|(J3\-n)|(J1\-n)", "Jan", row, flags = re.I)
      row = re.sub("1\-Q", "10", row, flags = re.I)
      row = re.sub("([A-Z]{3})$", r"\1-16", row, flags = re.I)
      row = re.sub("(Ja$)|(J$)", r"Jan-16", row, flags = re.I)
      row = re.sub("(\-J\-)|(\-n\-)", "-Jan-", row, flags = re.I)
      row = re.sub("\_\s*11\-11111\-114", "", row, flags = re.I)
      
      
#       row = re.sub("\-\d\s+\d+$", "14", row)
      row = re.sub("\-p\-", "-Apr-", row)
#       row = re.sub("([a-z]{3})\-\d-([a-z]{3})", r"\1-14 01-\2", row, flags = re.I)
#       row = re.sub("\s+\d+$", "", row)
      row = re.sub("¥", "", row)
      row = re.sub("yy+", "y", row, flags = re.I)
      row = re.sub("Ju1", "Jul", row, flags = re.I)
      row = re.sub("1¢J", "14", row, flags = re.I)
                  
      row = re.sub("0ct", "Oct", row, flags = re.I)
      row = re.sub("M0y", "May", row, flags = re.I)
      row = re.sub("N0v", "Nov", row, flags = re.I)
      row = re.sub("bn", r"Jan", row, flags = re.I)
#       row = re.sub("\d+\-\d+$", "", row)
      row = re.sub("Q4", "04", row)
      row = re.sub("1ug.*", "", row)
#       row = re.sub("(Ja0)|(JJan)", "Jan", row, flags = re.I)
      row = re.sub("JANN-.*", "", row, flags = re.I)
      row = re.sub("(\s+\-([a-z]{3}).*)|(\-JANNA.*)|(\s+\-\s+ep1\-5)|(\s+\d+$)|(\[)|(2\-a\_5.*)|(\sJ$)|(\s1\-0)|(\s1\-*u.*)|(\()", "", row, flags = re.I)
      
      row = re.sub("(\-GANET.*)|(\-\s*\-)|(\d\s*[a-z]+$)", "", row, flags = re.I)
      row = re.sub("(\s+J$)|(5-e□-.*)|(\-0\s+5\-Sep0.*)|()", "", row, flags = re.I)
      row = re.sub("e0", "Dec", row, flags = re.I)
      row = re.sub("(\-A(\d|n)\-)", "-Jan-", row, flags = re.I)
      row = re.sub("\s-(\d)", r'\1', row, flags = re.I)
      row = re.sub("(Au\d)|(Aua)", "Aug", row, flags = re.I)
      row = re.sub("22227", "222 27", row, flags = re.I)
      row = re.sub("(\d+\s5\-en)|(14\s5\-Dec)|(\d+\s5\-e.*)|(\d+\-en.*)", "", row, flags = re.I)
      row = re.sub("(Dec\-$)|(\d+\-0cJ5)", "", row, flags = re.I)
      row = re.sub("JJ", "J", row, flags = re.I)
      row = re.sub("J5", "15", row, flags = re.I)
      row = re.sub("2\-Q0", "20", row, flags = re.I)
      
#       row = re.sub("17\-D\-*$", "", row)
      
#       row = re.sub("(\d*ug.*)|(\sug.*)|(010\-U.*)|(\s22\s+1.*)|(A2.*)|(0U.*)", "", row, flags = re.I)
#       row = re.sub("(A\d.*)|(\s+ec.*)|(\s+u\d.*)|(\s+16\-UJANU.*)|(1ua.*)|(2\-J.*)|(31\-Dec\-\s+5\-e)|(1-Se)|(5\-$)|(03\-A)\
#         |(  173 23)|(uh.*)", "", row, flags = re.I)
      
      row = row.strip()
            
      if len(row) > 55  and\
        hasNumbers(row):
        
        print("--> Raw row:%s"%row)
        
#         Initially Last column is:8-Au
# Both date columns missing
# splitting again and the last column is 99-Jan-9999 and second last column is 8-Au
# second last column: 8-Au
  
        # Check if the second last columna is a date columns or not
        # In some cases the last date column is not read
        row_cols = row.split(' ')
        
        last_col = row_cols[len(row_cols)-1].strip()
        second_last_col = row_cols[len(row_cols)-2].strip()
        
#         print("Initially Last column is:{}".format(last_col))
        # if last column is not a date - add last and second last date columns
        if (not bool(re.match("\d{1,2}\-[a-z]{3}\-\d\d", last_col, flags = re.I))):
#           print("Both date columns missing")
          # append the available date column at the end of the row
          row = row + " 99-Jan-9999"
        
        row_cols = row.split(' ')
        
        last_col = row_cols[len(row_cols)-1].strip()
        second_last_col = row_cols[len(row_cols)-2].strip()
#         print("splitting again and the last column is {0} and second last column is {1}".format(last_col,second_last_col))
#         if (bool(re.match("\s*\d{1,2}\-[a-z]{3}\-\d{2,4}\s*", second_last_col, flags = re.I))):
#           print("second last column is a date")
                     
                     
        # Second last column is not a date but last column is
        if (not bool(re.match("\s*\d{1,2}\-[a-z]{3}\-\d{2,4}\s*", second_last_col, flags = re.I)) and\
                    bool(re.match("\s*\d{1,2}\-[a-z]{3}\-\d{2,4}\s*", last_col, flags = re.I))):
#           print("second last column: %s"%second_last_col)
          # append the available date column at the end of the row
          row = row + " "\
            + str(int(last_col.split("-")[0]) + 1) + "-"\
            + last_col.split("-")[1] + "-"\
            + last_col.split("-")[2]
          
      

        print(row)
        print("----------------------------------")

        global r
        r = row
        col_num = 0

        for col_name in col_names:
          col_val = get_col()
          cols[col_name] = col_val
          col_val = re.sub(r"([0-9]+)_", "\g<1>",col_val)
          col_val = re.sub(r"_([0-9]+)", "\g<1>",col_val)
          col_val = re.sub(r"\s+-([0-9]+)", " \g<1>",col_val)
          col_val = re.sub(r"([0-9]+)-\s+", "\g<1> ",col_val)

          # Write in spreadsheet
          cell = cells[col_num] + str(sheet_row_num)    
          sheet[cell].value= col_val
#           print("Col Name:{0}, col value:{1} written in cell {2}".format(col_name, col_val,cell))
          col_num = col_num + 1

        r = re.sub("\\s+", "_", r)
        cols["spool_no"] = r
        cell ="D"+str(sheet_row_num)    
        sheet[cell].value = r

        cell ="C"+str(sheet_row_num)    
        sheet[cell].value = page_num

        sheet_row_num = sheet_row_num + 1
  except Exception:
    print("Exception on page%i" %page_num)
    traceback.print_exc()
    pass
  
#   print("page_num:{0}, total_pages:{1}".format(page_num, total_pages))

  if(page_num == total_pages):
    break
          
  page_num = page_num + 1
  page_num_printed = page_num_printed + 1

wb.save("/content/gdrive/My Drive/Colab Notebooks/data/20files/L290-AQ-MDR-13850_0_SECTN_106OF107_IPX Reviewed_AC-OCR.xlsx")
        


Page#1
Page num printed#13322
--> Raw row:PA 007 15-Jan-WMP-2 7 147 143 104 107 124 125230 232 204 206 208 216 3-May-58 5-May-58
PA 007 15-Jan-WMP-2 7 147 143 104 107 124 125230 232 204 206 208 216 3-May-58 5-May-58
----------------------------------
--> Raw row:PA1-15-Jan-WMP-14 7 146 150 132 140 128 139 230 220 212 232 240 227 3-May-58
PA1-15-Jan-WMP-14 7 146 150 132 140 128 139 230 220 212 232 240 227 3-May-58 4-May-58
----------------------------------
Page#2
Page num printed#13323
--> Raw row:PA 009 15-Jan-WMP-2 7 110 128 126 138 140 128 246 220 218 236 240 232 3-May-58 06-Dec-14
PA 009 15-Jan-WMP-2 7 110 128 126 138 140 128 246 220 218 236 240 232 3-May-58 06-Dec-14
----------------------------------
--> Raw row:PA 009 15-Jan-WMP-3 7 144 142 134 120 116 131216 230 232 240 250 234 3-May-58 06-Dec-14
PA 009 15-Jan-WMP-3 7 144 142 134 120 116 131216 230 232 240 250 234 3-May-58 06-Dec-14
----------------------------------
Page#3
Page num printed#13324
Page#4
Page num printed#13325
P

In [0]:
re.search('\s+\-\d+$', 'A2-E1-PlO-14 1-WMP-16 7 110 109 124 122 120 117 208 220 222 219 240 222 7-Jun-15 9-Jun-14 -15')

In [0]:
! cp "/content/gdrive/My Drive/Inpex files /L290-AQ-MDR-13850_0_SECTN_87OF107_IPX Reviewed_AC-OCR.pdf" "/content/gdrive/My Drive/Colab Notebooks/data/20files/"
! cp "/content/gdrive/My Drive/Inpex files /L290-AQ-MDR-13850_0_SECTN_104OF107_IPX Reviewed_AC-OCR.pdf" "/content/gdrive/My Drive/Colab Notebooks/data/20files/"
! cp "/content/gdrive/My Drive/Inpex files /L290-AQ-MDR-13850_0_SECTN_105OF107_IPX Reviewed_AC-OCR.pdf" "/content/gdrive/My Drive/Colab Notebooks/data/20files/"
! cp "/content/gdrive/My Drive/Inpex files /L290-AQ-MDR-13850_0_SECTN_106OF107_IPX Reviewed_AC-OCR.pdf" "/content/gdrive/My Drive/Colab Notebooks/data/20files/"